# Leverage LLMs for Amazon Product Categorization

## Introduction

In this notebook, you'll learn how to leverage the native large language model (LLM) endpoints in Microsoft Fabric. The scenario is to use SynapseML and LangChain to build an LLM model to categorize Amazon products to relevant categories based on their name and description. 

The main steps in this notebook are:

1. Import and Install required libraries
2. Load the data
3. Leverage SynapseML and LangChain to create an LLM model
4. Demonstrate the model performance

#### Prerequisites

- In order to leverage LLM programming in Microsoft Fabric, you would require a paid Fabric capacity (F64 or higher). Read [here](https://aka.ms/fabric/copilot-capacity) about the capacity requirements.

- [Add a lakehouse](https://aka.ms/fabric/addlakehouse) to this notebook. You'll be downloading data from a public blob and storing the data in the lakehouse. 




## Step 1: Install and import required libraries

Before we move forward with categorization of Amazon products, it is imperative to first install LangChain and then import the essential libraries from LangChain, Spark, and SynapseML.

In [ ]:
%pip install openai==0.28.1 | grep -v 'already satisfied'

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 30, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [ ]:
%pip install openai langchain==0.0.331 | grep -v 'already satisfied'

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 36, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [ ]:
import os, openai, langchain, uuid
os.environ['OPENAI_API_VERSION'] = '2023-05-15'
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema.messages import HumanMessage, SystemMessage

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 38, Finished, Available)

## Step 2: Load the data

### Dataset

The dataset contains information about 7996 different products that are sold on Amazon. The products are represented by attributes such as `Product_Name`, `About_Product`, `Technical_Details`, `Shipping_Weight`, and `Product_Specification`.

### Download dataset and upload to lakehouse

> [!TIP]
> By defining the following parameters, you can use this notebook with different datasets easily.


In [ ]:
IS_CUSTOM_DATA = False  # if TRUE, dataset has to be uploaded manually

DATA_FOLDER = "Files/amazon-products"  # folder with data files
DATA_FILE = "amazon_products.csv"  # data file name

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 39, Finished, Available)

This code downloads a publicly available version of the dataset and then stores it in a Fabric lakehouse.

> [!IMPORTANT]
> **Make sure you [add a lakehouse](https://aka.ms/fabric/addlakehouse) to the notebook before running it. Failure to do so will result in an error.**

In [ ]:
if not IS_CUSTOM_DATA:

    import os, requests
    # Download demo data files into lakehouse if not exist
    remote_url = "https://synapseaisolutionsa.blob.core.windows.net/public/AmazonProducts"
    fname = "amazon_products.csv"
    download_path = f"/lakehouse/default/{DATA_FOLDER}/raw"

    if not os.path.exists("/lakehouse/default"):
        raise FileNotFoundError("Default lakehouse not found, please add a lakehouse and restart the session.")
    os.makedirs(download_path, exist_ok=True)
    if not os.path.exists(f"{download_path}/{fname}"):
        r = requests.get(f"{remote_url}/{fname}", timeout=30)
        with open(f"{download_path}/{fname}", "wb") as f:
            f.write(r.content)
    print("Downloaded demo data files into lakehouse.")

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 40, Finished, Available)

Downloaded demo data files into lakehouse.


### Read raw data from the lakehouse

Reads raw data from the **Files** section of the lakehouse.

In [ ]:
df_spark = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", True)
    .load(f"{DATA_FOLDER}/raw/{DATA_FILE}")
    .cache()
)

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 41, Finished, Available)

### Display raw data

Explore the raw data using the `display` command. For more information, see [Notebook visualization in Microsoft Fabric](https://aka.ms/fabric/visualization).

In [ ]:
display(df_spark)

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 42, Finished, Available)

SynapseWidget(Synapse.DataFrame, fc88e12e-8cd4-42e1-8005-e0f5e9b38859)

In [ ]:
# List the columns of the Spark DataFrame
df_spark.columns

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 43, Finished, Available)

['Product_Name',
 'About_Product',
 'Technical_Details',
 'Shipping_Weight',
 'Product_Specification']

In [ ]:
# Display DataFrame schema
df_spark.printSchema()

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 44, Finished, Available)

root
 |-- Product_Name: string (nullable = true)
 |-- About_Product: string (nullable = true)
 |-- Technical_Details: string (nullable = true)
 |-- Shipping_Weight: string (nullable = true)
 |-- Product_Specification: string (nullable = true)



## Step 3: Create the LLM model

Leverage SynapseML and LangChain to initialize a conversational agent that utilizes the specified GPT-3.5 model hosted on Azure to group Amazon products to relevant categories.

> [!TIP]
> You don't need to provide any subscription keys or reference any resource ID on Azure.

In [ ]:
llm = AzureChatOpenAI(
    deployment_name='gpt-35-turbo',
    model_name='gpt-35-turbo',
    temperature=0.1,
    verbose=False,
)

template = """
    Your job is to determine the product category.
    Please use all information available in the dataset to determine the product category as if this is going to be sold on Amazon.
    Provide multiple categories separated by a comma if multiple categories are approprate.
    If you are unsure or a category cannot be determined, say "Unknown".
    Write the category as a single word or short phrase.
    Examples:
    DC Cover Girls: Black Canary by Joëlle Jones Statue: Toys,
    Pacific Play Tent Agility Dog Training Chute: Pet Supplies."""

system_message = SystemMessage(content=template)
human_template= "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt)

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 45, Finished, Available)

## Step 4: Demonstrate the model performance

Create a small sample of the spark DataFrame to validate the performance.

In [ ]:
# Create a sample DataFrame
df_sample = df_spark.sample(False, 0.2, seed=0).limit(30)
display(df_sample)

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 46, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4d2f41df-63c3-4b45-a8ca-251d0f9c1ba4)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from synapse.ml.services.langchain import LangchainTransformer

transformer = (
    LangchainTransformer()
    .setInputCol("Product_Name")
    .setOutputCol("Product_Category")
    .setChain(chain)
)
display(transformer.transform(df_sample))

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 47, Finished, Available)

SynapseWidget(Synapse.DataFrame, 397ce9d1-0f1f-42a3-8055-dad7126026f4)

Save the new spark DataFrame that contains the product categories into the lakehouse.

In [ ]:
# Save the new spark DataFrame with product category into the lakehouse
df_sample.write.format("delta").mode("overwrite").save(f"{DATA_FOLDER}/df_sample_productCategory")

StatementMeta(, a4dd0a71-9384-41ec-bbf3-76fb7d5f449d, 48, Finished, Available)